In [3]:
# scrap PDF loop data from toby christie's site
# https://tobychristie.com/loop-data-2021-nascar-cup-series/"
# this is v2 of the scraper because there is a different pattern of PDF that tabula could output
# this scraper works for some of the races in 2022 cup series. (Discovered additional merged column patterns)

# skip to the last cell and execute, 
# if you get an error then go through each cell to see what is going on
# likely issues with the merged columns.

# tabula is the library that reads in searchable PDFs
import tabula
import pandas as pd
import re
from tabula import read_pdf                                                      
HEADERS_1=["Car","Driver","StartPos","MidRace","CloserPos","FinishPos","HighPos","LapsLed", "LowPos","AvgPos","PassDiff","GreenPass","GreenPassed","QualityPasses","PercentQualityPasses","NumFastestLaps","LapsTop15","PercentLapsTop15","PercentLapsLed", "TotalLaps","DriverRating","Pts"]
HEADERS_2=["Car","Driver","StartPos","MidRace","CloserPos","FinishPos","HighPos","LowPos","AvgPos","PassDiff","GreenPass","GreenPassed","QualityPasses","PercentQualityPasses","NumFastestLaps","LapsTop15","PercentLapsTop15","LapsLed","PercentLapsLed","TotalLaps","DriverRating","Pts"]
data_location=r".\SIADS591_592_Milestone1\Milestone_I\Milestone_I\data\loop_TobyChristie\pdf\2022\LD12211_POST_POSTBOOK.pdf"
df=read_pdf(data_location,pages='1')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\kchen\\OneDrive\\MADS\\SIADS591_592_Milestone1\\Milestone_I\\Milestone_I\\data\\loop_TobyChristie\\pdf\\2022\\LD12211_POST_POSTBOOK.pdf'

In [1]:
# preview uncleaned data
# df[0].head(10)

In [15]:
# extract the race date and race track names for csv output
fname_track=df[0]['Box Score'][0].replace(" ","")
date_text=df[0]['Box Score'][2]
pattern=r'(\d*)/(\d*)/(\d*)'
fname_year=re.search(pattern,date_text).group(3)
fname_day=re.search(pattern,date_text).group(2).zfill(2)
fname_month=re.search(pattern,date_text).group(1).zfill(2)


In [16]:
# dataframe cleanup steps:
# 1. removed extra header information
df2=df[0].iloc[6:]
# 2. handling merged columns 
df2['Box Score']=df2['Box Score'].str.split()
df2['Unnamed: 7']=df2['Unnamed: 7'].str.split()
df2['Unnamed: 9']=df2['Unnamed: 9'].str.split()

<ipython-input-16-1adc89defd34>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Box Score']=df2['Box Score'].str.split()
<ipython-input-16-1adc89defd34>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Unnamed: 7']=df2['Unnamed: 7'].str.split()
<ipython-input-16-1adc89defd34>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [2]:
# df2.head()

In [18]:
#split merged columns into the respected columns, this information can be gathered from the PDF.
df3=pd.DataFrame(df2['Box Score'].tolist(), columns=["LowPos","AvgPos","PassDiff","GreenPass","GreenPassed","QualityPasses","PercentQualityPasses","NumFastestLaps"])
df4=pd.DataFrame(df2['Unnamed: 7'].tolist(), columns=["LapsTop15","PercentLapsTop15"])
df5=pd.DataFrame(df2['Unnamed: 9'].tolist(), columns=["PercentLapsLed", "TotalLaps","DriverRating","Pts"])
# resetting the index to prep for the concat
df2=df2[['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6','Unnamed: 8']].reset_index(drop=True)
# combine the split up columns with the properly parsed columns
df_final=pd.concat([df2,df3,df4,df5],axis=1)
df_final.head()
df_final.columns=HEADERS_1
df_final=df_final[HEADERS_2]

In [21]:
# double checking
# df_final.head(2)

,Car,Driver,StartPos,MidRace,CloserPos,FinishPos,HighPos,LowPos,AvgPos,PassDiff,...,QualityPasses,PercentQualityPasses,NumFastestLaps,LapsTop15,PercentLapsTop15,LapsLed,PercentLapsLed,TotalLaps,DriverRating,Pts
0,9,Chase Elliott,4,8,1,1,1,20,4.1,11,...,32,96.97,33,399,99.8,73,18.3,400,132.8,50
1,47,Ricky Stenhouse Jr,15,16,2,2,2,19,8.9,21,...,52,88.14,8,376,94.0,0,0.0,400,102.8,41


In [23]:
# final output
df_final.to_csv(r'.\SIADS591_592_Milestone1\Milestone_I\Milestone_I\data\loop_TobyChristie\csv\2022\{}.csv'.format(fname_year+fname_month+fname_day+'_'+fname_track),index=False)


In [3]:
# put it all together and loopiong through a directory where the downloaded PDFs are.
import tabula
import pandas as pd
import re
from tabula import read_pdf

# loop through the whole directory
directory=r'.\SIADS591_592_Milestone1\Milestone_I\Milestone_I\data\loop_TobyChristie\pdf\2022'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        # read in data
        df=read_pdf(f,pages='1')
        # get the file name info
        fname_track=df[0]['Box Score'][0].replace(" ","")
        date_text=df[0]['Box Score'][2]
        pattern=r'(\d*)/(\d*)/(\d*)'
        fname_year=re.search(pattern,date_text).group(3)
        # zfill Fill the string with zeros until it is 2 characters long:
        fname_day=re.search(pattern,date_text).group(2).zfill(2)
        fname_month=re.search(pattern,date_text).group(1).zfill(2)
        # getting the data cleaned up
        print(f)
        df2=df[0].iloc[6:]
        df2['Box Score']=df2['Box Score'].str.split()
        df2['Unnamed: 9']=df2['Unnamed: 9'].str.split()
        df2['Unnamed: 11']=df2['Unnamed: 11'].str.split()

        # continue to clean
        df3=pd.DataFrame(df2['Box Score'].tolist(), columns=["AvgPos","PassDiff","GreenPass","GreenPassed","QualityPasses","PercentQualityPasses"])
        df4=pd.DataFrame(df2['Unnamed: 9'].tolist(), columns=["LapsTop15","PercentLapsTop15"])
        df5=pd.DataFrame(df2['Unnamed: 11'].tolist(), columns=["PercentLapsLed", "TotalLaps","DriverRating","Pts"])
        
        df2=df2[['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
        'Unnamed: 5', 'Unnamed: 6','Unnamed: 7', 'Unnamed: 8','Unnamed: 10']].reset_index(drop=True)
        
        # final cleaning steps
        df_final=pd.concat([df2,df3,df4,df5],axis=1)
        df_final.columns=HEADERS_1
        df_final=df_final[HEADERS_2]

        # output the data
        df_final.to_csv(r'.\SIADS591_592_Milestone1\Milestone_I\data\loop_TobyChristie\csv\2021\{}.csv'.format(fname_year+fname_month+fname_day+'_'+fname_track),index=False)


C:\Users\kchen\OneDrive\MADS\SIADS591_592_Milestone1\Milestone_I\Milestone_I\data\loop_TobyChristie\pdf\2022\LD12211_POST_POSTBOOK.pdf


<ipython-input-3-53202170564b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Box Score']=df2['Box Score'].str.split()
<ipython-input-3-53202170564b>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Unnamed: 9']=df2['Unnamed: 9'].str.split()


KeyError: 'Unnamed: 11'